<a href="https://colab.research.google.com/github/AdamPeetz/stable_diffusion/blob/main/image_enhancing_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import os
import time
from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [16]:
SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1"

**Defining Helper Functions**

In [21]:
def load_image(image_path):
  input_image = tf.image.decode_image(tf.io.read_file(image_path))
  if input_image.shape[-1] == 4:
    input_image = input_image[...,:-1]
  input_image = tf.cast(input_image, tf.float32)
  return tf.expand_dims(input_image,0)

In [22]:
directory_path = "/content/gdrive/My Drive/planegan/sample_output_2/"
image_lst = []
for file in os.listdir(directory_path):
  add_image = load_image(os.path.join(directory_path,file))
  image_lst.append(add_image)

In [ ]:
image_lst[1]

#### Performing Super Resolution of images loaded from path

In [17]:
model = hub.load(SAVED_MODEL_PATH)

Downloaded https://tfhub.dev/captain-pool/esrgan-tf2/1, Total size: 20.60MB



In [ ]:
start = time.time()
counter = 0
for idx in range(len(image_lst)):
  fake_image = model(image_lst[idx])
  fake_image = tf.squeeze(fake_image)
  image = np.asarray(fake_image)
  image = tf.clip_by_value(image, 0, 255)
  image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
  plt.figure(figsize=(20,20))
  plt.imshow(image)
  plt.axis('off')
  filename = '/content/gdrive/My Drive/planegan/sample_output_2/generated_plot_%05d.png' % (counter+1)
  plt.savefig(filename, transparent=True,bbox_inches="tight",pad_inches=0.0 )
  plt.close()
  counter += 1
print("Time Taken: %f" % (time.time() - start))